In [55]:
import os
import json
import pandas as pd
import numpy as np
import jsonlines

vital_path = 'demo_data'
#load all BVP,HR,SpO2
    
def find_csv_files(directory):
    csv_files = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('BVP.csv') or file.endswith('HR.csv') or file.endswith('SpO2.csv'):
                csv_files.append(os.path.join(root, file))
    
    return csv_files


In [62]:
vital_csv_files = find_csv_files(vital_path)
vital_csv_files.sort()
vital_csv_files

['demo_data\\light_1\\v01\\BVP.csv',
 'demo_data\\light_1\\v01\\HR.csv',
 'demo_data\\light_1\\v01\\SpO2.csv',
 'demo_data\\light_1\\v02\\BVP.csv',
 'demo_data\\light_1\\v02\\HR.csv',
 'demo_data\\light_1\\v02\\SpO2.csv',
 'demo_data\\light_2\\v01\\BVP.csv',
 'demo_data\\light_2\\v01\\HR.csv',
 'demo_data\\light_2\\v01\\SpO2.csv',
 'demo_data\\light_2\\v02\\BVP.csv',
 'demo_data\\light_2\\v02\\HR.csv',
 'demo_data\\light_2\\v02\\SpO2.csv',
 'demo_data\\light_3\\v01\\BVP.csv',
 'demo_data\\light_3\\v01\\HR.csv',
 'demo_data\\light_3\\v01\\SpO2.csv',
 'demo_data\\light_3\\v02\\BVP.csv',
 'demo_data\\light_3\\v02\\HR.csv',
 'demo_data\\light_3\\v02\\SpO2.csv',
 'demo_data\\light_4\\v01\\BVP.csv',
 'demo_data\\light_4\\v01\\HR.csv',
 'demo_data\\light_4\\v01\\SpO2.csv',
 'demo_data\\light_4\\v02\\BVP.csv',
 'demo_data\\light_4\\v02\\HR.csv',
 'demo_data\\light_4\\v02\\SpO2.csv',
 'demo_data\\light_5\\v01\\BVP.csv',
 'demo_data\\light_5\\v01\\HR.csv',
 'demo_data\\light_5\\v01\\SpO2.csv',
 

In [63]:
#save the data in text format
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import glob

HR_dict = {}
SpO2_dict = {}
BVP_dict = {}
csv_files = vital_csv_files
for file in csv_files:
    #spilt the file name by BVP,HR,SpO2
    if file.endswith('HR.csv'):
        data = pd.read_csv(file)['hr']
        #save the data in int num format then to the dictionary,split with ,
        data_text = list(map(int, data))
        HR_dict[file] = data_text
    elif file.endswith('SpO2.csv'):
        data = pd.read_csv(file)['spo2']
        data_text = list(map(int, data))
        SpO2_dict[file] = data_text
    elif file.endswith('BVP.csv'):
        data = pd.read_csv(file)['bvp']
        data_text = list(map(int, data))
        BVP_dict[file] = data_text




In [66]:
#chunk the vitals for 60 points
def chunk_vitals(vitals,length=60):
    vital_list = []
    for i in range(0, len(vitals), length):
        if i+length > len(vitals):
            break
            vital_list.append(vitals[i:])
        else:
            vital_list.append(vitals[i:i+60])
    return vital_list

vitals_test =SpO2_dict['demo_data\\light_6\\v01\\SpO2.csv']
vitals_chunks = chunk_vitals(vitals_test)
for vital in vitals_chunks:
    print(len(vital))


60
60
60


In [109]:
prompt_HR ={'introduction':'Here is a list of Heart Rate data of myself. Each point refers to a second.','task':'Please tell me the  average heart rate. Keep one decimal place. And give me analysis of health. If the heart rate for anytime is under 100, health state is 0. If the  heart rate for anytime is 100-130, health state is 1. If the heart rate is above 130, health state is 2. Then tell me why you have such judgment on the reasons part. Please consider the trend of the vital in time series as well. Please output as the format required: The average heart rate : XXX. The health state : XXX. Suggestions: XXX. Resons: XXX.','background':'An ideal heart rate is between 50 and 90 beats per minute (bpm). It is acceptable to continue home monitoring for 101-109. If the heart rate is 110 to 130, you would better seek advice from your GP. If the heart rate is 140 or above, you should seek medical advice immediately.','output_format':'The average heart rate : XXX. The health state : XXX. Suggestions: XXX. Resons: XXX.'}
prompt_SpO2 ={'introduction':'Here is a list of Blood Oxygen(SpO2 value) data of myself. Each point refers to a second.','task':'Please tell me the average blood oxygen. Keep one decimal place. And give me analysis of health. If SpO2 for anytim is between 96% and 99%, health state is 0. If SpO2 for anytim is between 93% and 95%, health state is 1. If SpO2 for anytim is 92 or less, health state is 2. Then tell me why you have such judgment on the reasons part. Please consider the trend of the vital in time series as well. Please output as the format required: The average blood oxygen : XXX. The health state : XXX. Suggestions: XXX. Resons: XXX.','background':'A normal blood oxygen level varies between between 96% and 99% in healthy individuals.It is acceptable to continue home monitoring for 93%-95%. If the blood oxygen level is 92% or less, it is considered low and called hypoxemia,you would better seek advice from your GP. If the blood oxygen level is below 85%, it is considered critical and called hypoxia, you should seek medical advice immediately.','output_format':'The average blood oxygen level : XXX. The health state : XXX. Suggestions: XXX. Reasons: XXX.'}
prompt_All = {'introduction':'Here is a list of Heart Rate data and a list of Blood Oxygen(SpO2 value) data  of myself. Each point refers to a second.','task':'Please tell me the average blood oxygen and the average heart rate. Keep one decimal place. And give me analysis of health. Only two vitals is normal then the health is normal and output 0. Any abnormal of any vital should be considered abnormal and output 1. Then tell me why you have such judgment on the reasons part. Please consider the trend of the vital in time series as well. Please output as the format required: The average heart rate : XXX. The average blood oxygen : XXX. The health state : XXX. Suggestions: XXX. Resons: XXX.','background':'A normal blood oxygen level varies between between 96% and 99% in healthy individuals.It is acceptable to continue home monitoring for 93%-95%. If the blood oxygen level is 92% or less, it is considered low and called hypoxemia,you would better seek advice from your GP. If the blood oxygen level is below 85%, it is considered critical and called hypoxia, you should seek medical advice immediately. If SpO2 for anytim is between 96% and 99%, health state is 0. If SpO2 for anytim is between 93% and 95%, health state is 1. If SpO2 for anytim is 92 or less, health state is 2. An ideal heart rate is between 50 and 90 beats per minute (bpm). It is acceptable to continue home monitoring for 101-109. If the heart rate is 110 to 130, you would better seek advice from your GP. If the heart rate is 140 or above, you should seek medical advice immediately. If the  heart rate for anytime is under 100, health state is 0. If the  heart rate for anytime is 100-130, health state is 1. If the  heart rate for anytime is above 130, health state is 2. ','output_format':'The average heart rate : XXX. The average blood oxygen : XXX. The health state : XXX. Suggestions: XXX. Reasons: XXX.'}
# prompt_BVP = {'introduction':'Here is a list of Blood Volume Pulse data of myself. Each second refers to 20 points.','task':'Please tell me the average blood volume pulse of this subject. And give me analysis of health of the subject.'}

In [106]:
def generate_gt(vitals,label='HR'):
    #label = 'HR' or 'SpO2' or 'BVP'
    #vitals = vitals_hr or vitals_spo2 or vitals_bvp
    #both return: health state, 0: normal, 1: abnormal, 2: extreme abnormal
    #HR：return average HR and the max HR
    #SpO2：return average SpO2 and the min SpO2
    #BVP：return average HR 
    health_state = 0
    if label=='HR':
        average = np.mean(vitals)
        max_v = np.max(vitals)
        if max_v>=100:
            health_state = 1
        if max_v>130:
            health_state = 2
        return health_state,average,max_v
    elif label=='SpO2':
        average = np.mean(vitals)
        min_v = np.min(vitals)
        if min_v<=95:
            health_state = 1
        if min_v<=92:
            health_state = 2
        return health_state,average,min_v
    elif label=='BVP':
        average = np.mean(vitals)
        if average>0.5:
            health_state = 1
        return health_state,average

        


In [103]:
#save ground truth to csv file ['gt_hr','gt_hr_health','gt_spo2','gt_spo2_health','gt_health'] 
chunk_len = 60
csv_file_path = 'result_len'+str(chunk_len)+'.csv'
csv_file = open(csv_file_path, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['gt_hr','gt_hr_health','gt_spo2','gt_spo2_health','gt_health'])
file_list = vital_csv_files
for file in file_list:
    if file.endswith('HR.csv'):
        gt_health = 0
        file1 = file
        file2 = file[:-6]+'SpO2.csv'
        print('file name:',file1,file2)
        vitals1 = HR_dict[file1]
        vitals2 = SpO2_dict[file2]
        vitals_chunks1 = chunk_vitals(vitals1,chunk_len)
        vitals_chunks2 = chunk_vitals(vitals2,chunk_len)
        for chunk1,chunk2 in zip(vitals_chunks1,vitals_chunks2):
            gt1 = generate_gt(chunk1,'HR')
            gt2 = generate_gt(chunk2,'SpO2')
            if gt1[0]!=0 or gt2[0]!=0:
                gt_health = 1
            csv_writer.writerow([gt1[1],gt1[0],gt2[1],gt2[0],gt_health])
csv_file.close()



file name: demo_data\light_1\v01\HR.csv demo_data\light_1\v01\SpO2.csv
file name: demo_data\light_1\v02\HR.csv demo_data\light_1\v02\SpO2.csv
file name: demo_data\light_2\v01\HR.csv demo_data\light_2\v01\SpO2.csv
file name: demo_data\light_2\v02\HR.csv demo_data\light_2\v02\SpO2.csv
file name: demo_data\light_3\v01\HR.csv demo_data\light_3\v01\SpO2.csv
file name: demo_data\light_3\v02\HR.csv demo_data\light_3\v02\SpO2.csv
file name: demo_data\light_4\v01\HR.csv demo_data\light_4\v01\SpO2.csv
file name: demo_data\light_4\v02\HR.csv demo_data\light_4\v02\SpO2.csv
file name: demo_data\light_5\v01\HR.csv demo_data\light_5\v01\SpO2.csv
file name: demo_data\light_5\v02\HR.csv demo_data\light_5\v02\SpO2.csv
file name: demo_data\light_6\v01\HR.csv demo_data\light_6\v01\SpO2.csv
file name: demo_data\light_6\v02\HR.csv demo_data\light_6\v02\SpO2.csv


In [107]:
#generate ground truth and text data
def gt_text(file_list,mode='All',chunk_len=60):
    #mode:'All'-HR+SpO2
    #mode:'HR'-HR
    #mode:'SpO2'-SpO2
    #input: file name list
    #output: ground truth and text data to LLM

    if mode=='HR':
        for file in file_list:
            if file.endswith('HR.csv'):
                print('file name:',file)
                vitals = HR_dict[file]
                vitals_chunks = chunk_vitals(vitals,chunk_len)
                for chunk in vitals_chunks:
                    gt = generate_gt(chunk,'HR')
                    text = str(prompt_HR)+'\n'+'HR data :'+str(chunk)
                    print('ground truth:',gt)
                    print('text:',text)
                    print('---------------------------')
    elif mode=='SpO2':
        for file in file_list:
            if file.endswith('SpO2.csv'):
                print('file name:',file)
                vitals = SpO2_dict[file]
                vitals_chunks = chunk_vitals(vitals,chunk_len)
                for chunk in vitals_chunks:
                    gt = generate_gt(chunk,'SpO2')
                    text = str(prompt_SpO2)+'\nSpO2 data :'+str(chunk)
                    print('ground truth:',gt)
                    print('text:',text)
                    print('---------------------------')
    elif mode=='All':
        for file in file_list:
            if file.endswith('HR.csv'):
                file1 = file
                file2 = file[:-6]+'SpO2.csv'
                print('file name:',file1,file2)
                vitals1 = HR_dict[file1]
                vitals2 = SpO2_dict[file2]
                vitals_chunks1 = chunk_vitals(vitals1,chunk_len)
                vitals_chunks2 = chunk_vitals(vitals2,chunk_len)
                for chunk1,chunk2 in zip(vitals_chunks1,vitals_chunks2):
                    gt1 = generate_gt(chunk1,'HR')
                    gt2 = generate_gt(chunk2,'SpO2')
                    gt = 'HR: '+str(gt1)+'\n SpO2:'+str(gt2)
                    text = str(prompt_All)+'\n HR data :'+str(chunk1)+'\n SpO2 data :'+str(chunk2)
                    print('ground truth:',gt)
                    print('text:',text)
                    print('---------------------------')
    


In [110]:
gt_text(csv_files,'SpO2',60)

file name: demo_data\light_1\v01\SpO2.csv
ground truth: (0, 97.93333333333334, 97)
text: {'introduction': 'Here is a list of Blood Oxygen(SpO2 value) data of myself. Each point refers to a second.', 'task': 'Please tell me the average blood oxygen. Keep one decimal place. And give me analysis of health. If SpO2 for anytim is between 96% and 99%, health state is 0. If SpO2 for anytim is between 93% and 95%, health state is 1. If SpO2 for anytim is 92 or less, health state is 2. Then tell me why you have such judgment on the reasons part. Please consider the trend of the vital in time series as well. Please output as the format required: The average blood oxygen : XXX. The health state : XXX. Suggestions: XXX. Resons: XXX.', 'background': 'A normal blood oxygen level varies between between 96% and 99% in healthy individuals.It is acceptable to continue home monitoring for 93%-95%. If the blood oxygen level is 92% or less, it is considered low and called hypoxemia,you would better seek ad

In [1]:
#calculate results from csv file,  MAE and MAPE for regression, accuracy for classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

csv_file = 'results/result_len60.csv'
df = pd.read_csv(csv_file)
df

c:\Users\T JACK\.conda\envs\OxfordAI_1_20220118\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\T JACK\.conda\envs\OxfordAI_1_20220118\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\T JACK\.conda\envs\OxfordAI_1_20220118\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,gt_hr,gt_hr_health,gt_spo2,gt_spo2_health,gt_health,pred_hr,pred_hr_health,pred_spo2,pred_spo2_health,pred_all_hr,pred_all_spo2,pred_health
0,77.516667,0,97.933333,0,0,77.9,0,98.00,0,77.0,98.0,0
1,79.233333,0,95.616667,1,1,79.2,0,96.20,1,79.4,96.2,1
2,81.850000,0,91.266667,2,1,81.8,1,91.70,2,82.0,91.5,1
3,77.016667,0,98.166667,0,0,77.1,0,98.50,0,77.1,98.4,0
4,70.266667,0,96.966667,0,0,70.9,0,97.20,0,71.5,97.1,1
5,72.383333,0,95.033333,2,1,71.1,0,95.00,1,72.8,95.5,1
6,67.616667,0,94.133333,2,1,67.6,0,94.50,1,68.0,74.5,1
7,71.033333,0,97.633333,0,0,70.9,0,97.70,0,71.5,97.8,0
8,77.066667,0,92.200000,2,1,77.1,0,92.20,2,76.8,92.8,1
9,66.416667,0,97.050000,1,1,66.2,0,97.20,0,67.3,97.1,1


In [2]:
#calculate MAE and MAPE for hr and spo2
# groundtruth: hr:gt_hr, spo2:gt_spo2
# prediction: hr:pred_hr for single run,pred_all_hr for mixed run; spo2:pred_spo2 for single run,pred_all_spo2 for mixed run
# output: MAE and MAPE for hr and spo2 for single run and mixed run
# input: gt_hr,gt_spo2,pred_hr,pred_spo2,pred_all_hr,pred_all_spo2
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error,mean_squared_error,f1_score,accuracy_score,precision_score,recall_score,roc_auc_score
gt_hr,gt_spo2,pred_hr,pred_spo2,pred_all_hr,pred_all_spo2 = df['gt_hr'],df['gt_spo2'],df['pred_hr'],df['pred_spo2'],df['pred_all_hr'],df['pred_all_spo2']
hr_single_mae = mean_absolute_error(gt_hr,pred_hr)
hr_single_mape = mean_absolute_percentage_error(gt_hr,pred_hr)
hr_mixed_mae = mean_absolute_error(gt_hr,pred_all_hr)
hr_mixed_mape = mean_absolute_percentage_error(gt_hr,pred_all_hr)
spo2_single_mae = mean_absolute_error(gt_spo2,pred_spo2)
spo2_single_mape = mean_absolute_percentage_error(gt_spo2,pred_spo2)
spo2_mixed_mae = mean_absolute_error(gt_spo2,pred_all_spo2)
spo2_mixed_mape = mean_absolute_percentage_error(gt_spo2,pred_all_spo2)
hr_single_rmse = mean_squared_error(gt_hr,pred_hr,squared=False)
hr_mixed_rmse = mean_squared_error(gt_hr,pred_all_hr,squared=False)
spo2_single_rmse = mean_squared_error(gt_spo2,pred_spo2,squared=False)
spo2_mixed_rmse = mean_squared_error(gt_spo2,pred_all_spo2,squared=False)

#save two decimal places
print('hr_single_mae:',round(hr_single_mae,2))
print('hr_single_mape:',format(hr_single_mape, ".2%"))
print('hr_mixed_mae:',round(hr_mixed_mae,2))
print('hr_mixed_mape:',format(hr_mixed_mape, ".2%"))
print('spo2_single_mae:',round(spo2_single_mae,2))
print('spo2_single_mape:',format(spo2_single_mape, ".2%"))
print('spo2_mixed_mae:',round(spo2_mixed_mae,2))
print('spo2_mixed_mape:',format(spo2_mixed_mape, ".2%"))
print('hr_single_rmse:',round(hr_single_rmse,2))
print('hr_mixed_rmse:',round(hr_mixed_rmse,2))
print('spo2_single_rmse:',round(spo2_single_rmse,2))
print('spo2_mixed_rmse:',round(spo2_mixed_rmse,2))




<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


hr_single_mae: 0.29
hr_single_mape: 0.40%
hr_mixed_mae: 0.58
hr_mixed_mape: 0.82%
spo2_single_mae: 0.25
spo2_single_mape: 0.26%
spo2_mixed_mae: 0.79
spo2_mixed_mape: 0.84%
hr_single_rmse: 0.39
hr_mixed_rmse: 0.74
spo2_single_rmse: 0.32
spo2_mixed_rmse: 3.29


In [3]:
#calculate the accuracy and F1 score for hr_health, spo2_health, and health
#prediction: hr:pred_hr_health(0,1,2), spo2:pred_spo2_health(0,1,2), health:pred_health(0,1)
#true label: hr:gt_hr_health(0,1,2), spo2:gt_spo2_health(0,1,2), health(0,1):if gt_hr_health and spo2_health are both 0, then gt_health=0, otherwise gt_health=1
#input: the predicted label and the true label
#output: the accuracy and F1 score

gt_hr_health,pred_hr_health,gt_spo2_health,pred_spo2_health,gt_health,pred_health = df['gt_hr_health'],df['pred_hr_health'],df['gt_spo2_health'],df['pred_spo2_health'],df['gt_health'],df['pred_health']
#calculate gt health form gt_hr_health and gt_spo2_health

acc_hr_health = accuracy_score(gt_hr_health, pred_hr_health)
acc_spo2_health = accuracy_score(gt_spo2_health, pred_spo2_health)
acc_health = accuracy_score(gt_health, pred_health)
f1_hr_health = f1_score(gt_hr_health, pred_hr_health, average='macro')
f1_spo2_health = f1_score(gt_spo2_health, pred_spo2_health, average='macro')
f1_health = f1_score(gt_health, pred_health, average='macro')
print('acc_hr_health:',acc_hr_health,'f1_hr_health:',f1_hr_health)
print('acc_spo2_health:',acc_spo2_health,'f1_spo2_health:',f1_spo2_health)
print('acc_health:',acc_health,'f1_health:',f1_health)





acc_hr_health: 0.9722222222222222 f1_hr_health: 0.4929577464788732
acc_spo2_health: 0.6111111111111112 f1_spo2_health: 0.6176142697881829
acc_health: 0.8333333333333334 f1_health: 0.7923076923076924


In [5]:
#FOR gt_spo2_health and pred_spo2_health, change any value above 0 to 1

gt_spo2_health = gt_spo2_health.replace(to_replace = [2], value = 1)
pred_spo2_health = pred_spo2_health.replace(to_replace = [2], value = 1)

0     0
1     1
2     1
3     0
4     0
5     1
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    0
17    1
18    1
19    0
20    1
21    1
22    1
23    1
24    0
25    1
26    1
27    0
28    1
29    0
30    1
31    1
32    1
33    1
34    1
35    1
Name: gt_spo2_health, dtype: int64

In [6]:
acc_hr_health = accuracy_score(gt_hr_health, pred_hr_health)
acc_spo2_health = accuracy_score(gt_spo2_health, pred_spo2_health)
acc_health = accuracy_score(gt_health, pred_health)
f1_hr_health = f1_score(gt_hr_health, pred_hr_health, average='macro')
f1_spo2_health = f1_score(gt_spo2_health, pred_spo2_health, average='macro')
f1_health = f1_score(gt_health, pred_health, average='macro')
print('acc_hr_health:',acc_hr_health,'f1_hr_health:',f1_hr_health)
print('acc_spo2_health:',acc_spo2_health,'f1_spo2_health:',f1_spo2_health)
print('acc_health:',acc_health,'f1_health:',f1_health)

acc_hr_health: 0.9722222222222222 f1_hr_health: 0.4929577464788732
acc_spo2_health: 0.6666666666666666 f1_spo2_health: 0.49896480331262943
acc_health: 0.8333333333333334 f1_health: 0.7923076923076924
